<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title"><b>Steel Production Planning via Gurobi</b></span> by <a xmlns:cc="http://creativecommons.org/ns#" href="http://mate.unipv.it/gualandi" property="cc:attributionName" rel="cc:attributionURL">Stefano Gualandi</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>. Based on a project at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/mathcoding/opt4ds" rel="dct:source">https://github.com/mathcoding/opt4ds</a>.


## 2.1 Solving Steel Production Planning

In this notebook, we explain how to solve the **Linear Programming** problem that we have written to solve the Steel Planning problem during the class (see the slides on KIRO). This problem is given as Exercise 1.1 in Chapter 1 of [Linear Programming, Foundations and Extensions](https://link.springer.com/book/10.1007/978-1-4614-7630-6) by [R.J. Vanderbei](https://vanderbei.princeton.edu/).

We show below how to use [Gurobi](https://www.gurobi.com/academia/academic-program-and-licenses/) to define the **variables**, the **objective function**, and the **constraints**.

### 2.1.1 Software Installation
First, we need to install the [Gurobi](http://www.gurobi.org/). If you are running this notebook in a Colab, you don't need to install anything else on your computer.

The following line installs the free version of Gurobi in a Google Colab or on you computer.

In [ ]:
# Run if on Colab
%pip install gurobipy

To install an academic licence of Gurobi on your computer follow these [instructions](https://www.gurobi.com/features/academic-named-user-license/).

### 2.1.2 Define the data
Recall, that a possible model of the steel planning problem is as follows:

$$
\begin{align}
\max \quad & p_B x_B + p_C x_C  \\
 \quad & \frac{x_B}{r_B} + \frac{x_C}{r_C} \leq T \\
& 0 \leq x_B \leq d_B\\
& 0 \leq x_C \leq d_C
\end{align}
$$

Hence, first, we need to define the data of our instance:

In [1]:
# Data
pB = 25     # Profit of band ($ profit per tons)
pC = 30     # Profit of coil ($ profit per tonr)

rB = 200    # Production rate of band (tons per hour)
rC = 140    # Production rate of coil (tons per hour)

dB = 6000   # Maximum demand for band (per tons)
dC = 4000   # Maximum demand for coil (per tons)

T = 40      # Total hours available (per week)

### 2.1.3 Define the model entities
To build the Linear Programming model with Gurobi, we need first to import the gurobi library:

In [2]:
from gurobipy import Model, GRB

At this point, we first declare a global object that refer to our model, creating an instance of the class [Model](https://docs.gurobi.com/projects/optimizer/en/current/reference/python/model.html#Model):

In [3]:
model = Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-16


Notice the `Model` is a python class, and we are initializing an object called `model` of type `Model`.

Then, we declare the two nonnegative variables, along with their cost coefficients and type, using the [model.addVar](https://docs.gurobi.com/projects/optimizer/en/current/reference/python/model.html#Model.addVar) method:

In [4]:
# Declare the decision variables
xB = model.addVar(lb=0, ub=dB, obj=pB, vtype=GRB.CONTINUOUS, name='xB')
xC = model.addVar(lb=0, ub=dC, obj=pC, vtype=GRB.CONTINUOUS, name='xC')

Here, we add variable `xB` and `xC` to the model. The two variables are of type `GRB.CONTINUOUS`, and, we are declaring the two variables $0 \leq x_B \leq U_B$, $0 \leq x_C \leq U_c$. The cost coefficients are `pB` and `pC`.

We need to specify a direction for the objective function: min or max

In [5]:
model.ModelSense = GRB.MAXIMIZE

Next step is to introduce the linear constraint using the [model.addConstr](https://docs.gurobi.com/projects/optimizer/en/current/reference/python/model.html#Model.addConstr) method:

In [6]:
# Declare constraints
model.addConstr(1/rB * xB + 1/rC * xC <= T)

<gurobi.Constr *Awaiting Model Update*>

In [7]:
# To finalize the model call:
model.update()

Notice that we are **declaring** the model, without programming any algorithm to actually solve this model. To find the optimal solution of this LP.

### 2.1.4 Solve the model
We have use the gurobi python library to *declare* our Linear Programming model. Next, we use the gurobi solver actually find the optimal values for the decision variables.

In [8]:
# Solver call
model.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 2 columns and 2 nonzeros
Model fingerprint: 0x9c2d724e
Coefficient statistics:
  Matrix range     [5e-03, 7e-03]
  Objective range  [2e+01, 3e+01]
  Bounds range     [4e+03, 6e+03]
  RHS range        [4e+01, 4e+01]
Presolve time: 0.00s
Presolved: 1 rows, 2 columns, 2 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0000000e+05   1.250000e+02   0.000000e+00      0s
       1    1.9200000e+05   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.920000000e+05


Every time we invoke a solver, it is very good practice to check the status of the solver, since it may have stop its execution for several different reasons:

In [9]:
# Basic info about the solution process
print(f"Status: {model.Status}, ObjVal: {model.ObjVal}")

Status: 2, ObjVal: 192000.0


Be aware of the meaning of the different status code, by checking the [Optimization Status Code](https://docs.gurobi.com/projects/optimizer/en/current/reference/numericcodes/statuscodes.html#optimization-status-codes).

Whenever the status of the solver is `2=OPTIMAL`, you can query the solver to get the values of the decision variables.

In [10]:
# Report solution value
print("Decision variables:")
print("\tProduction of bands:", xB.X)
print("\tProduction of coils:", xC.X)

Decision variables:
	Production of bands: 6000.0
	Production of coils: 1400.0


**REMARK:** To check the problem actually solved by Gurobi the standard LP format, you can write the following command:

In [ ]:
model.write('production.lp')

To check where the file is written:

In [ ]:
!ls

### 2.1.5 Complete Script
We report below the complete script.

In [12]:
# Import the library
from gurobipy import Model, GRB

# Create an instance of the model object
model = Model()

# Declare the decision variables
xB = model.addVar(lb=0, ub=dB, obj=pB, vtype=GRB.CONTINUOUS, name='xB')
xC = model.addVar(lb=0, ub=dC, obj=pC, vtype=GRB.CONTINUOUS, name='xC')

# Specify the objective function direction
model.ModelSense = GRB.MINIMIZE

# Declare the single linear constraint
model.addConstr(1/rB * xB + 1/rC * xC <= T)

# Solver call
model.optimize()

# Basic info about the solution process
print(f"Status: {model.Status}, ObjVal: {model.ObjVal}")

# Report value of the decision variables
print("Decision variables:")
print("\tProduction of bands:", xB.X)
print("\tProduction of coils:", xC.X)

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 2 columns and 2 nonzeros
Model fingerprint: 0xb536d1b6
Coefficient statistics:
  Matrix range     [5e-03, 7e-03]
  Objective range  [2e+01, 3e+01]
  Bounds range     [4e+03, 6e+03]
  RHS range        [4e+01, 4e+01]
Presolve removed 1 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  0.000000000e+00
Status: 2, ObjVal: 0.0
Decision variables:
	Production of bands: 0.0
	Production of coils: 0.0
